# C2M2 Assessment

We perform a file-centric FAIR Assessment on all files defined in a [C2M2 datapackage](https://docs.nih-cfde.org/en/latest/c2m2/draft-C2M2_specification/) according to the [C2M2 Rubric](https://fairshake.cloud/rubric/36); descriptions of each metric and how we assess them are provided below, along with the actual code to perform the assesssment.

In [1]:
import os
import re
import glob
import requests
import numpy as np
import pandas as pd
import itertools as it
from matplotlib import pyplot as plt

from textwrap import dedent
from IPython.display import display, Markdown

In [2]:
def one_and_only(it):
  ''' Select one and only item from an iterable, otherwise throw an exception.
  '''
  it = iter(it)
  ret = next(it)
  try:
    next(it)
    raise Exception('Expected one')
  except StopIteration:
    return ret

def deep_find(root, file):
  ''' Helper for finding a filename in a potentially deep directory
  '''
  return set(glob.glob(os.path.join(root, '**', file), recursive=True))

def fetch_cache(url, filename, cachedir='.cached'):
  ''' Download a {file} from a {url} if it hasn't already been downloaded, storing it in {cachedir}.
  '''
  import os, urllib.request
  os.makedirs(cachedir, exist_ok=True)
  if not os.path.exists(os.path.join(cachedir, filename)):
    urllib.request.urlretrieve(url, filename=os.path.join(cachedir, filename))
  return os.path.join(cachedir, filename)

def url_join(*args):
  ''' Join urls by slashes, not worrying about duplicated trailing slashes
  '''
  return '/'.join([arg.rstrip('/') for arg in args[:-1]]+[args[-1]])

def filter_empty(val):
  ''' Attempt to catch some actual null values that aren't really null.
  '''
  return [
    v
    for v in val
    if v is not None and (
      type(v) != str or v.strip().lower() not in {
        '-',
        '-666',
        '',
        'empty',
        'n/a',
        'na',
        'nan',
        'nil',
        'none',
        'not defined',
        'null',
        'undef',
        'undefined',
      }
    )
  ]


## Step 1. Load DERIVA compatible client from URL or datapackage

Given a datapackage, access it through DERIVA-compatible client. This client package <https://github.com/nih-cfde/deriva-datapackage> permits accessing offline datapackages in the same way that the online DERIVA client operates, thus the assessment can be performed online or offline.

In [3]:
import zipfile
import tempfile
file = "/Users/maayanlab/Documents/CFDE/04-2022/Archive.zip"
basename, ext = os.path.splitext(file)
assert ext == '.zip', 'Expected .zip file'
directory = tempfile.mkdtemp()
with zipfile.ZipFile(file, 'r') as z:
  z.extractall(directory)

In [4]:
from deriva_datapackage import create_offline_client

# sometimes zip files zip the leading directory, which may be named anything,
#  deep_find lets us locate the datapackage wherever it is.
CFDE = create_offline_client(
    *(
      deep_find(directory, 'C2M2_datapackage.json')
      | deep_find(directory, 'datapackage.json')
    ),
    cachedir=directory,
)

/Users/maayanlab/opt/anaconda3/envs/cfde_venv/lib/python3.8/site-packages/tabulator/stream.py:420: UserWarning: Not supported option(s) "delimiter, doublequote, lineterminator, quotechar, skipinitialspace" for scheme "file" and format "tsv"
  warnings.warn(message, UserWarning)


## Step 2. Compute and report global metrics about the datapackage

### Table 1: Entity Counts

In [5]:
total_files = CFDE.tables['file'].count()
total_collections = CFDE.tables['collection'].count()
total_biosamples = CFDE.tables['biosample'].count()
total_subjects = CFDE.tables['subject'].count()
total_projects = CFDE.tables['project'].count()
entity_counts = pd.Series({
  name: table.count()
  for name, table in CFDE.tables.items()
}).to_frame('Entity Counts')
entity_counts.to_csv('entity-counts.tsv', sep='\t')
entity_counts

,Entity Counts
analysis_type,0
anatomy,60
assay_type,28
biosample,1466838
biosample_disease,1211130
biosample_from_subject,1466838
biosample_gene,361735
biosample_in_collection,12002
biosample_substance,860327
collection,409


## Step 3. Compute and report metric compliance for the datapackage

We use a python decorator for registering each metric into the rubric. This lets us define each metric in its own cell with its description and code to assert it. Because of the nature of these assertions, assessments are performed as metrics are registered. This assessment is datapackage-centric rather than file centric.

This paradigm can be used for any rubric allowing assessment code to remain the same even with changing metrics, furthermore this is compatible with [FAIRshake](https://fairshake.cloud/) assessments, adopting FAIRshake metric identifiers allowing the results to be easily registered with FAIRshake.

In [6]:
rubric = {
  '@id': 36,
  'name': 'NIH CFDE Interoperability',
  'description': 'This rubric identifies aspects of the metadata models which promote interoperable dataset querying and filtering',
  'metrics': {},
}
answers = []

def _rbInterpolate(value):
  if pd.isna(value): return f"#666666"
  r, g, b = int(255. * (1-value)), 0, int(255. * value)
  return f"{r:02x}{g:02x}{b:02x}"

def _asPct(value):
  if pd.isna(value):
    return "NaN"
  else:
    return f"{value*100:.2f}%"

def _register_metric_answer(schema):
  global metrics
  global answers
  def wrapper(func):
    rubric['metrics'][schema['@id']] = dict(schema, func=func)
    link = '' if schema['@id'] < 0 else f" ([{schema['@id']}](https://fairshake.cloud/metric/{schema['@id']}))"
    display(Markdown(dedent(f'''
      ### Metric{link}: {schema['name']}
      **{schema['description']}**

      {schema['detail']}
    ''')))
    answer = dict(func(), metric=schema['@id'])
    answers.append(answer)
    display(Markdown(dedent(f'''
      #### Results: <span style="font-weight: bold; color: #{_rbInterpolate(answer['value'])}">{_asPct(answer['value'])}</span> '''
    ) + dedent(answer['comment'])))
  setattr(wrapper, '__name__', schema['name'])
  return wrapper

In [7]:
@_register_metric_answer({
  # standardized metadata format (107), machine readable metadata (106)
  # metadata license (117) (c2m2 ?)
  '@id': 106,
  'name': 'Metadata conformance',
  'description': 'The metadata properly conforms with the CFDE perscribed metadata model specification',
  'detail': '''The average metadata coverage of all tables''',
  'principle': 'Findable',
})
def _():
  display(Markdown('### Table 2. Metadata Coverage'))
  coverage = pd.DataFrame(
    dict(
      table=table_name,
      coverage=len(list(filter_empty(entity.values()))) / len(table.column_definitions.keys()),
    )
    for table_name, table in CFDE.tables.items()
    for entity in table.entities()
  ).groupby('table')['coverage'].describe().fillna(0).sort_values('mean')
  coverage.to_csv('coverage.tsv', sep='\t')
  display(coverage)
  value = coverage['mean'].mean()
  return {
    'value': value,
    'comment': f'(See metadata coverage for more info)',
  }


### Metric ([106](https://fairshake.cloud/metric/106)): Metadata conformance
**The metadata properly conforms with the CFDE perscribed metadata model specification**

The average metadata coverage of all tables


### Table 2. Metadata Coverage

,count,mean,std,min,25%,50%,75%,max
table,,,,,,,,
subject,1966.0,0.577823,5.827496e-02,0.500000,0.500000,0.600000,0.600000,0.700
collection,409.0,0.678484,6.314685e-02,0.500000,0.625000,0.625000,0.750000,0.750
project,16.0,0.732143,1.463850e-01,0.571429,0.571429,0.714286,0.857143,1.000
biosample,1466838.0,0.748706,1.267034e-02,0.625000,0.750000,0.750000,0.750000,0.875
compound,29315.0,0.750000,0.000000e+00,0.750000,0.750000,0.750000,0.750000,0.750
file,1454802.0,0.791869,2.731600e-02,0.700000,0.800000,0.800000,0.800000,0.800
biosample_from_subject,1466838.0,0.800000,5.551117e-16,0.800000,0.800000,0.800000,0.800000,0.800
substance,29437.0,0.800000,0.000000e+00,0.800000,0.800000,0.800000,0.800000,0.800
disease,29.0,0.991379,4.642383e-02,0.750000,1.000000,1.000000,1.000000,1.000



#### Results: <span style="font-weight: bold; color: #1100ed">93.13%</span> (See metadata coverage for more info)

In [8]:
# TODO: Help us identify other acceptible globally unique persistent and valid identifiers
@_register_metric_answer({
  # Persistent identifier (105)
  '@id': 104,
  'name': 'Persistent identifier',
  'description': 'Globally unique, persistent, and valid identifiers (preferrably DOIs) are present for the dataset',
  'detail': '''We check that the persistent id that are present are DOIs.''',
  'principle': 'Findable',
})
def _():
  qualified_persistent_ids = pd.Series({
    (file['id_namespace'], file['local_id'], file.get('persistent_id')): 1 if file.get('persistent_id') and re.match(r'^https?://[^/]+\.doi\.org/.+$', file['persistent_id']) else 0
    for file in CFDE.tables['file'].entities()
  }).sort_values()
  display(qualified_persistent_ids)
  total_qualified_persistent_ids = qualified_persistent_ids.sum()
  value = (total_qualified_persistent_ids / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f'({total_qualified_persistent_ids} / {total_files})',
  }


### Metric ([104](https://fairshake.cloud/metric/104)): Persistent identifier
**Globally unique, persistent, and valid identifiers (preferrably DOIs) are present for the dataset**

We check that the persistent id that are present are DOIs.


https://www.lincsproject.org/  LDS-1505                                                                  http://lincsportal.ccs.miami.edu/datasets/view/LDS-1505                                                                                   0
                               L1000_LINCS_DCIC_2021_MOAR008_NCIH2110_24H_B17_flunarizine_3.33uM.tsv.gz  https://lincs-dcic.s3.amazonaws.com/LINCS-data-2020/L1000/compound/L1000_LINCS_DCIC_MOAR008_NCIH2110_24H_B17_flunarizine_3.33uM.tsv.gz    0
                               L1000_LINCS_DCIC_2021_MOAR008_NCIH2110_24H_B16_flunarizine_10uM.tsv.gz    https://lincs-dcic.s3.amazonaws.com/LINCS-data-2020/L1000/compound/L1000_LINCS_DCIC_MOAR008_NCIH2110_24H_B16_flunarizine_10uM.tsv.gz      0
                               L1000_LINCS_DCIC_2021_MOAR008_NCIH2110_24H_B15_AH-6809_1.11uM.tsv.gz      https://lincs-dcic.s3.amazonaws.com/LINCS-data-2020/L1000/compound/L1000_LINCS_DCIC_MOAR008_NCIH2110_24H_B15_AH-6809_1.11uM.tsv.gz        0
                    


#### Results: <span style="font-weight: bold; color: #ff0000">0.00%</span> (0 / 1454802)

In [9]:
@_register_metric_answer({
  '@id': -1,
  'name': 'ratio files are associated with data type term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_data_type = CFDE.tables['data_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].data_type == CFDE.tables['data_type'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_data_type / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_data_type} / {total_files})",
  }


### Metric: ratio files are associated with data type term
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1454802 / 1454802)

In [10]:
@_register_metric_answer({
  '@id': -2,
  'name': 'ratio files are associated with file format term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_file_format = CFDE.tables['file_format'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].file_format == CFDE.tables['file_format'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_file_format / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_file_format} / {total_files})",
  }


### Metric: ratio files are associated with file format term
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1454802 / 1454802)

In [11]:
@_register_metric_answer({
  '@id': -3,
  'name': 'ratio files are associated with assaytype term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_assay_type = CFDE.tables['assay_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].assay_type == CFDE.tables['assay_type'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_assay_type / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_assay_type} / {total_files})",
  }


### Metric: ratio files are associated with assaytype term
****





#### Results: <span style="font-weight: bold; color: #0000fe">100.00%</span> (1454799 / 1454802)

In [12]:
@_register_metric_answer({
  '@id': -4,
  'name': 'ratio files are associate with anatomy term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_biosample'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_biosample'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_anatomy / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_anatomy} / {total_files})",
  }


### Metric: ratio files are associate with anatomy term
****





#### Results: <span style="font-weight: bold; color: #0200fc">98.96%</span> (1439684 / 1454802)

In [13]:
@_register_metric_answer({
  '@id': -5,
  'name': 'ratio files are associated with a biosample',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_biosample = CFDE.tables['biosample'] \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_biosample'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_biosample'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_biosample / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_biosample} / {total_files})",
  }


### Metric: ratio files are associated with a biosample
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1454802 / 1454802)

In [14]:
@_register_metric_answer({
  '@id': -6,
  'name': 'ratio files are associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_subject'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_subject'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_subject / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_subject} / {total_files})",
  }


### Metric: ratio files are associated with a subject
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1454802 / 1454802)

In [15]:
@_register_metric_answer({
  '@id': -7,
  'name': 'ratio files are associated with a subject_role_taxonomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_subject_role = CFDE.tables['subject_role_taxonomy'] \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_subject'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_subject'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_associated_with_subject_role / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_associated_with_subject_role} / {total_files})",
  }


### Metric: ratio files are associated with a subject_role_taxonomy
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1454802 / 1454802)

In [16]:
@_register_metric_answer({
  '@id': -8,
  'name': 'ratio biosamples are associated with a species term (NCBI Taxon)',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_ncbi_taxon = CFDE.tables['ncbi_taxonomy'] \
    .link(CFDE.tables['subject_role_taxonomy'], on=(
      CFDE.tables['subject_role_taxonomy'].taxonomy_id == CFDE.tables['ncbi_taxonomy'].id
    )) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_from_subject'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_from_subject'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_associated_with_ncbi_taxon / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_associated_with_ncbi_taxon} / {total_biosamples})",
  }


### Metric: ratio biosamples are associated with a species term (NCBI Taxon)
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1466838 / 1466838)

In [17]:
@_register_metric_answer({
  '@id': -9,
  'name': 'ratio biosamples are associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_from_subject'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_from_subject'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_associated_with_subject / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_associated_with_subject} / {total_biosamples})",
  }


### Metric: ratio biosamples are associated with a subject
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1466838 / 1466838)

In [18]:
@_register_metric_answer({
  '@id': -10,
  'name': 'ratio biosamples are associated with a file',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['file_describes_biosample'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['file_describes_biosample'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_associated_with_file / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_associated_with_file} / {total_biosamples})",
  }


### Metric: ratio biosamples are associated with a file
****





#### Results: <span style="font-weight: bold; color: #0000fe">99.99%</span> (1466665 / 1466838)

In [19]:
@_register_metric_answer({
  '@id': -11,
  'name': 'ratio biosamples are associated with an anatomy term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_associated_with_anatomy / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_associated_with_anatomy} / {total_biosamples})",
  }


### Metric: ratio biosamples are associated with an anatomy term
****





#### Results: <span style="font-weight: bold; color: #0200fc">98.96%</span> (1451648 / 1466838)

In [20]:
@_register_metric_answer({
  '@id': -12,
  'name': 'ratio biosamples are associated with an assay term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_assay = CFDE.tables['file'].filter(CFDE.tables['file'].assay_type != None)   .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['file_describes_biosample'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['file_describes_biosample'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_associated_with_assay / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_associated_with_assay} / {total_biosamples})",
  }


### Metric: ratio biosamples are associated with an assay term
****





#### Results: <span style="font-weight: bold; color: #0000fe">99.97%</span> (1466452 / 1466838)

In [21]:
@_register_metric_answer({
  '@id': -13,
  'name': 'ratio subjects are associated with a taxonomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_taxonomy = CFDE.tables['ncbi_taxonomy'] \
    .link(CFDE.tables['subject_role_taxonomy'], on=(
      CFDE.tables['subject_role_taxonomy'].taxonomy_id == CFDE.tables['ncbi_taxonomy'].id
    )) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = (total_subjects_associated_with_taxonomy / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_associated_with_taxonomy} / {total_subjects})",
  }


### Metric: ratio subjects are associated with a taxonomy
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1966 / 1966)

In [22]:
@_register_metric_answer({
  '@id': -14,
  'name': 'ratio subjects have subject granularity',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_granularity = CFDE.tables['subject'].filter(
    (CFDE.tables['subject'].granularity != None) & (CFDE.tables['subject'].granularity != '')
  ).count()
  value = (total_subjects_associated_with_granularity / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_associated_with_granularity} / {total_subjects})",
  }


### Metric: ratio subjects have subject granularity
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1966 / 1966)

In [23]:
@_register_metric_answer({
  '@id': -15,
  'name': 'ratio subjects have taxonomic role',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_role_taxonomy = CFDE.tables['subject_role_taxonomy'] \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = (total_subjects_associated_with_role_taxonomy / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_associated_with_role_taxonomy} / {total_subjects})",
  }


### Metric: ratio subjects have taxonomic role
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1966 / 1966)

In [24]:
@_register_metric_answer({
  '@id': -16,
  'name': 'ratio subjects associated with a biosample',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_biosample = CFDE.tables['biosample'] \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['biosample_from_subject'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['biosample_from_subject'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = (total_subjects_associated_with_biosample / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_associated_with_biosample} / {total_subjects})",
  }


### Metric: ratio subjects associated with a biosample
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1966 / 1966)

In [25]:
@_register_metric_answer({
  '@id': -17,
  'name': 'ratio subjects associated with a file',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['file_describes_subject'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['file_describes_subject'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = (total_subjects_associated_with_file / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_associated_with_file} / {total_subjects})",
  }


### Metric: ratio subjects associated with a file
****





#### Results: <span style="font-weight: bold; color: #0000fe">99.69%</span> (1960 / 1966)

In [26]:
@_register_metric_answer({
  '@id': -18,
  'name': 'IF there are collections: # of files that are part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['file_in_collection'], on=((
      CFDE.tables['file_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['file_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_in_collection'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_in_collection'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = (total_files_not_in_collection / total_files) if total_files else float('nan')
  return {
    'value': value,
    'comment': f"({total_files_not_in_collection} / {total_files})",
  }


### Metric: IF there are collections: # of files that are part of a collection
****





#### Results: <span style="font-weight: bold; color: #fe0000">0.03%</span> (465 / 1454802)

In [27]:
@_register_metric_answer({
  '@id': -19,
  'name': 'IF there are collections: # of subjects that are part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['subject_in_collection'], on=((
      CFDE.tables['subject_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['subject_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_in_collection'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_in_collection'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = (total_subjects_not_in_collection / total_subjects) if total_subjects else float('nan')
  return {
    'value': value,
    'comment': f"({total_subjects_not_in_collection} / {total_subjects})",
  }


### Metric: IF there are collections: # of subjects that are part of a collection
****





#### Results: <span style="font-weight: bold; color: #4d00b1">69.48%</span> (1366 / 1966)

In [28]:
@_register_metric_answer({
  '@id': -20,
  'name': 'IF there are collections: # of biosamples that are part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['biosample_in_collection'], on=((
      CFDE.tables['biosample_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['biosample_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_in_collection'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_in_collection'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = (total_biosamples_not_in_collection / total_biosamples) if total_biosamples else float('nan')
  return {
    'value': value,
    'comment': f"({total_biosamples_not_in_collection} / {total_biosamples})",
  }


### Metric: IF there are collections: # of biosamples that are part of a collection
****





#### Results: <span style="font-weight: bold; color: #fc0002">0.82%</span> (12002 / 1466838)

In [29]:
@_register_metric_answer({
  '@id': -21,
  'name': 'Project associated with anatomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['biosample'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['biosample'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = (total_projects_associated_with_anatomy / total_projects) if total_projects else float('nan')
  return {
    'value': value,
    'comment': f"({total_projects_associated_with_anatomy} / {total_projects})",
  }


### Metric: Project associated with anatomy
****





#### Results: <span style="font-weight: bold; color: #1f00df">87.50%</span> (14 / 16)

In [30]:
@_register_metric_answer({
  '@id': -22,
  'name': 'Project associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['file'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['file'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = (total_projects_associated_with_file / total_projects) if total_projects else float('nan')
  return {
    'value': value,
    'comment': f"({total_projects_associated_with_file} / {total_projects})",
  }


### Metric: Project associated with files
****





#### Results: <span style="font-weight: bold; color: #1f00df">87.50%</span> (14 / 16)

In [31]:
@_register_metric_answer({
  '@id': -23,
  'name': 'Project associated with data types',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_data_type = CFDE.tables['data_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].data_type == CFDE.tables['data_type'].id
    )) \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['file'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['file'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = (total_projects_associated_with_data_type / total_projects) if total_projects else float('nan')
  return {
    'value': value,
    'comment': f"({total_projects_associated_with_data_type} / {total_projects})",
  }


### Metric: Project associated with data types
****





#### Results: <span style="font-weight: bold; color: #1f00df">87.50%</span> (14 / 16)

In [32]:
@_register_metric_answer({
  '@id': -24,
  'name': 'Project associated with subjects',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['subject'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['subject'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = (total_projects_associated_with_subject / total_projects) if total_projects else float('nan')
  return {
    'value': value,
    'comment': f"({total_projects_associated_with_subject} / {total_projects})",
  }


### Metric: Project associated with subjects
****





#### Results: <span style="font-weight: bold; color: #df001f">12.50%</span> (2 / 16)

In [33]:
@_register_metric_answer({
  '@id': -25,
  'name': 'list of any anatomy terms in anatomy.tsv NOT associated with biosamples',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # TODO: we can probably use an outer join for this..
  used_anatomy_terms = CFDE.tables['biosample'] \
      .link(CFDE.tables['anatomy'], on=(
          CFDE.tables['anatomy'].id == CFDE.tables['biosample'].anatomy
      )) \
      .groupby(CFDE.tables['anatomy'].id)
  used_anatomy_ids = {
      anatomy['id']
      for anatomy in used_anatomy_terms.entities()
  }
  unused_anatomy_terms = pd.DataFrame({
      anatomy['id']: anatomy
      for anatomy in CFDE.tables['anatomy'].entities()
      if anatomy['id'] not in used_anatomy_ids
  }).T
  unused_anatomy_terms.to_csv('unused-anatomy-terms.tsv', sep='\t')
  display(Markdown('\n'.join(f'''- {v['id']}''' for _, v in unused_anatomy_terms.iterrows())))
  total = len(used_anatomy_ids) + unused_anatomy_terms.shape[0]
  value = (1 - (unused_anatomy_terms.shape[0] / total)) if total else float('nan')
  return {
    'value': value,
    'comment': f"({len(used_anatomy_ids)} / {total})",
  }


### Metric: list of any anatomy terms in anatomy.tsv NOT associated with biosamples
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (60 / 60)

In [34]:
@_register_metric_answer({
  '@id': -26,
  'name': 'list of any species terms in ncbi_taxonomy.tsv NOT associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # TODO: we can probably use an outer join for this..
  used_taxonomy_terms = CFDE.tables['subject'] \
      .link(CFDE.tables['subject_role_taxonomy'], on=((
          CFDE.tables['subject_role_taxonomy'].subject_id_namespace == CFDE.tables['subject'].id_namespace
      ) & (
          CFDE.tables['subject_role_taxonomy'].subject_local_id == CFDE.tables['subject'].local_id
      ))) \
      .link(CFDE.tables['ncbi_taxonomy'], on=(
          CFDE.tables['ncbi_taxonomy'].id == CFDE.tables['subject_role_taxonomy'].taxonomy_id
      )) \
      .groupby(CFDE.tables['ncbi_taxonomy'].id)
  used_taxonomy_ids = {
      taxonomy['id']
      for taxonomy in used_taxonomy_terms.entities()
  }
  unused_taxonomy_terms = pd.DataFrame({
      taxonomy['id']: taxonomy
      for taxonomy in CFDE.tables['ncbi_taxonomy'].entities()
      if taxonomy['id'] not in used_taxonomy_ids
  }).T
  unused_taxonomy_terms.to_csv('unused-taxonomy-terms.tsv', sep='\t')
  display(Markdown('\n'.join(f'''- {v['id']}''' for _, v in unused_taxonomy_terms.iterrows())))
  total = len(used_taxonomy_ids) + unused_taxonomy_terms.shape[0]
  value = (1 - (unused_taxonomy_terms.shape[0] / total)) if total else float('nan')
  return {
    'value': value,
    'comment': f"({len(used_taxonomy_ids)} / {total})",
  }


### Metric: list of any species terms in ncbi_taxonomy.tsv NOT associated with a subject
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (1 / 1)

In [35]:
@_register_metric_answer({
  '@id': -27,
  'name': 'list of any assay terms in assay_type.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_assay_type_terms = CFDE.tables['file'] \
      .link(CFDE.tables['assay_type'], on=(
          CFDE.tables['assay_type'].id == CFDE.tables['file'].assay_type
      )) \
      .groupby(CFDE.tables['assay_type'].id)
  used_assay_type_ids = {
      assay_type['id']
      for assay_type in used_assay_type_terms.entities()
  }
  unused_assay_type_terms = pd.DataFrame({
      assay_type['id']: assay_type
      for assay_type in CFDE.tables['assay_type'].entities()
      if assay_type['id'] not in used_assay_type_ids
  }).T
  unused_assay_type_terms.to_csv('unused-assay-type-terms.tsv', sep='\t')
  display(Markdown('\n'.join(f'''- {v['id']}''' for _, v in unused_assay_type_terms.iterrows())))
  total = len(used_assay_type_ids) + unused_assay_type_terms.shape[0]
  value = (1 - (unused_assay_type_terms.shape[0] / total)) if total else float('nan')
  return {
    'value': value,
    'comment': f"({len(used_assay_type_ids)} / {total})",
  }


### Metric: list of any assay terms in assay_type.tsv NOT associated with files
****




- OBI:0200198


#### Results: <span style="font-weight: bold; color: #0900f5">96.43%</span> (27 / 28)

In [36]:
@_register_metric_answer({
  '@id': -28,
  'name': 'list of any format terms in file_format.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_file_format_terms = CFDE.tables['file'] \
      .link(CFDE.tables['file_format'], on=(
          CFDE.tables['file_format'].id == CFDE.tables['file'].file_format
      )) \
      .groupby(CFDE.tables['file_format'].id)
  used_file_format_ids = {
      file_format['id']
      for file_format in used_file_format_terms.entities()
  }
  unused_file_format_terms = pd.DataFrame({
      file_format['id']: file_format
      for file_format in CFDE.tables['file_format'].entities()
      if file_format['id'] not in used_file_format_ids
  }).T
  unused_file_format_terms.to_csv('unused-file-format-terms.tsv', sep='\t')
  display(Markdown('\n'.join(f'''- {v['id']}''' for _, v in unused_file_format_terms.iterrows())))
  total = len(used_file_format_ids) + unused_file_format_terms.shape[0]
  value = (1 - (unused_file_format_terms.shape[0] / total)) if total else float('nan')
  return {
    'value': value,
    'comment': f"({len(used_file_format_ids)} / {total})",
  }


### Metric: list of any format terms in file_format.tsv NOT associated with files
****




- format:3989


#### Results: <span style="font-weight: bold; color: #3f00bf">75.00%</span> (3 / 4)

In [37]:
@_register_metric_answer({
  '@id': -29,
  'name': 'list of any data type terms in data_type.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_data_type_terms = CFDE.tables['file'] \
      .link(CFDE.tables['data_type'], on=(
          CFDE.tables['data_type'].id == CFDE.tables['file'].data_type
      )) \
      .groupby(CFDE.tables['data_type'].id)
  used_data_type_ids = {
      data_type['id']
      for data_type in used_data_type_terms.entities()
  }
  unused_data_type_terms = pd.DataFrame({
      data_type['id']: data_type
      for data_type in CFDE.tables['data_type'].entities()
      if data_type['id'] not in used_data_type_ids
  }).T
  unused_data_type_terms.to_csv('unused-data-type-terms.tsv', sep='\t')
  display(Markdown('\n'.join(f'''- {v['id']}''' for _, v in unused_data_type_terms.iterrows())))
  total = len(used_data_type_ids) + unused_data_type_terms.shape[0]
  value = (1 - (unused_data_type_terms.shape[0] / total)) if total else float('nan')
  return {
    'value': value,
    'comment': f"({len(used_data_type_ids)} / {total})",
  }


### Metric: list of any data type terms in data_type.tsv NOT associated with files
****





#### Results: <span style="font-weight: bold; color: #0000ff">100.00%</span> (8 / 8)

In [38]:
@_register_metric_answer({
  '@id': 145,
  'name': 'Landing Page',
  'description': 'A landing page exists and is accessible for the identifiers',
  'detail': '''Checks to make sure the persistent_id is resolvable with a HEAD request. if it is not http/https it is assumed to be an identifiers.org-resolvable CURIE. note that this is still error prone, some identifier websites do not follow HTTP standards and may not report 404s with ids that aren't found.''',
  'principle': 'Findable',
})
def _():
  results = {}
  for file in CFDE.tables['file'].entities():
    file_id = (file['id_namespace'], file['local_id'])
    results[file_id] = {}
    persistent_id = file.get('persistent_id')
    if not persistent_id:
      results[file_id]['value'] = 0
      results[file_id]['comment'] = "No persistent id present"
      continue
    if not re.match(r'^https?://', persistent_id):
      persistent_id = 'https://identifiers.org/{}'.format(persistent_id)
    results[file_id]['value'] = float('nan')
    results[file_id]['comment'] = "Skipped"
  results = pd.DataFrame(results).T
  results.to_csv('landing-pages.tsv', sep='\t')
  display(results)
  display(results.describe())
  display(results['comment'].value_counts().to_frame('comment'))
  return {
    'value': results['value'].mean(),
    'comment': f'(based on status_code reports via HEAD)',
  }


### Metric ([145](https://fairshake.cloud/metric/145)): Landing Page
**A landing page exists and is accessible for the identifiers**

Checks to make sure the persistent_id is resolvable with a HEAD request. if it is not http/https it is assumed to be an identifiers.org-resolvable CURIE. note that this is still error prone, some identifier websites do not follow HTTP standards and may not report 404s with ids that aren't found.


value  comment
https://www.lincsproject.org/ LDS-1505             NaN  Skipped
                              LDS-1110             NaN  Skipped
                              LDS-1507             NaN  Skipped
                              LDS-1190             NaN  Skipped
                              LDS-1260             NaN  Skipped
...                                                ...      ...
                              MDD_ATACseq_Level3   NaN  Skipped
                              MDD_IF_Level3        NaN  Skipped
                              MDD_RPPA_Level3      NaN  Skipped
                              MDD_cycIF_Level3     NaN  Skipped
                              MDD_L1000_Level3     NaN  Skipped

[1454802 rows x 2 columns]

,value,comment
count,0,1454802
unique,0,1
top,NaN,Skipped
freq,NaN,1454802


,comment
Skipped,1454802



#### Results: <span style="font-weight: bold; color: ##666666">NaN</span> (based on status_code reports via HEAD)

## Step 4. Review results

With the assessment complete, we're ready to review the results.

### Table 3. A simple look at the structure of the answers dataframe (joined with metrics for readability of metrics)

 - `name` this is the human readable name of the metric
 - `principle` this is the F.A.I.R category of the metric
 - `value` represents the quantitative value assigned to the given answer. It ranges between 0.0 and 1.0, 0.0 representing complete lack of *compliance* with a metric, and 1.0 representing complete satisfaction of a metric.
 - `comment` is a human-description describing why the `value` is what it is.

In [39]:
df_metrics = pd.DataFrame(rubric['metrics']).T
df_answers = pd.merge(
    left=pd.DataFrame(answers), left_on='metric',
    right=df_metrics[['name', 'principle']], right_index=True,
)[['name', 'principle', 'value', 'comment']].sort_values('value')
df_answers.to_csv('summary.tsv', sep='\t')
df_answers

,name,principle,value,comment
1,Persistent identifier,Findable,0.000000,(0 / 1454802)
19,IF there are collections: # of files that are ...,,0.000320,(465 / 1454802)
21,IF there are collections: # of biosamples that...,,0.008182,(12002 / 1466838)
25,Project associated with subjects,,0.125000,(2 / 16)
20,IF there are collections: # of subjects that a...,,0.694812,(1366 / 1966)
29,list of any format terms in file_format.tsv NO...,,0.750000,(3 / 4)
23,Project associated with files,,0.875000,(14 / 16)
22,Project associated with anatomy,,0.875000,(14 / 16)
24,Project associated with data types,,0.875000,(14 / 16)
0,Metadata conformance,Findable,0.931303,(See metadata coverage for more info)


## Step 5. FAIRshake

Our assessment is now ready to be registered with FAIRshake.

In [40]:
def render_insignia(scores, metrics):
    from IPython.display import display, HTML
    import uuid, json
    id = str(uuid.uuid4())
    display(HTML(f'''
        <div id={repr(id)} style="width: 100px; height: 100px;"></div>
        <script>
        require(['https://fairshake.cloud/v2/static/scripts/insignia.js'], function(insignia) {{
            var metrics = {json.dumps(metrics)}
            var el = document.getElementById({repr(id)})
            for (var i = 0; i < el.children.length; i++) el.removeChild(el.children[i])
            insignia.build_svg(el,
                {json.dumps(scores)},
                {{ tooltips: function (rubric, metric, score) {{ return `${{(score*100).toFixed(0)}}%<br />${{metrics[metric]}}` }} }}
            )
        }})
        </script>
    '''), display_id=id)



### Figure 1. FAIRshake Insignia

The insignia compactly shows the results of the assessments; hovering over each square shows what metric was being evaluated. Blue means high satisfaction while red means low satisfaction.

In [41]:
render_insignia(
    { 0: df_answers['value'].to_dict() },
    df_answers['name'].to_dict()
)

## Cleanup

No need to run this locally, but useful for appyter.

In [ ]:
import shutil
shutil.rmtree(directory)